## Sentiment Analysis - Model Building
### Built specifically for cekmedsos.com

Storage for tweet data is stored on cekmedsos.com, communicating through database layer

### Training Data retrieved from:
https://github.com/ridife/dataset-idsa


In [1]:
# Imports
import pymysql.cursors
import pandas as pd
import numpy as np
import uuid
import time
import os
import string
import re
import tensorflow as tf


from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from datetime import datetime
from sklearn.model_selection import train_test_split

## need to install this !!!!
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Activation, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow import keras


# database connection properties
db_host='202.157.176.225'
db_user ='cekmedsos_db'
db_password='kuku838485*#'
db_database='cekmedsos_database'

# save model to what filename?
model_data_filename = 'train-data-01.h5'

# model parameters
num_of_epoch = 16

In [2]:
# Connect to the database
connection = pymysql.connect(host=db_host,
                             user=db_user,
                             password=db_password,
                             database=db_database,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

# get available jobs from database server, first come first serve
# sql = "select id, hastag, `parameter` from screen_analisis_ai where active = 1 and status = 1 and jenis_analisa = 1 order by created asc, id asc limit 1"
sql = "select id, hastag, `parameter` \
from screen_analisis_ai \
where active = 1 \
and status = 1 \
and jenis_analisa = 2 \
order by created asc, id asc limit 1"

row_count = cursor.execute(sql)

if(row_count == 0):
    # get out, nothing to do
    print('Zero jobs, quitting now')
    quit()

result = cursor.fetchall()
database_keyword_id = result[0]['hastag']
similarity_treshold = result[0]['parameter']
i_process_id = result[0]['id']
screen_name = ''

print(database_keyword_id)
print(similarity_treshold)
print(i_process_id)

#pigaihinasukujawa
0.85
693


## Query tweet data from database
using parameter retrieve from job list

In [3]:
# Connect to the database
connection = pymysql.connect(host='202.157.176.225',
                             user='cekmedsos_db',
                             password='kuku838485*#',
                             database='cekmedsos_database',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

# s_query_string = 'select id, tweet, tweet_date_time from ret_tweet where '
s_query_string = 'select id, tweet, tweet_date_time from vw_ret_tweet_clean where '

if (screen_name != ''):
    # print('use screen name')
    s_query_string = s_query_string + 'screen_name = "' + screen_name + '" and db_id = "' + str(database_keyword_id) + '"'
else:
    # print('no use')
    s_query_string = s_query_string + 'keyword = "' + database_keyword_id.replace('"','') + '" limit 25'
    
print(s_query_string)
df = pd.read_sql(s_query_string, con=connection)

# Close Connection
connection.close()

# see result
df.head()

select id, tweet, tweet_date_time from vw_ret_tweet_clean where keyword = "#pigaihinasukujawa" limit 25


,id,tweet,tweet_date_time
0,1354579947801759748,Jangan mancing di air keruh... #PigaiHinaSuk...,2021-01-28 07:00:01
1,1354609891395182592,Kritik itu baik untuk pemerintah. Kritik itu ...,2021-01-28 08:59:00
2,1354610143015673856,Natalius Pigai ngajak gelut cah... #PigaiHin...,2021-01-28 09:00:00
3,1354610143019888644,""" Natalius Pigai telah menghina suku Jawa dan ...",2021-01-28 09:00:00
4,1354610143032451072,Apa yang kalian tangkap dari pernyataan Pigai ...,2021-01-28 09:00:00


In [4]:
# function stemming
import nltk
from sklearn.pipeline import Pipeline
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# nltk.download('stopwords')

def stemming(comment):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in comment:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    return d_clean
    
# function case folding
import re
def casefolding(comment):
    comment = comment.lower()
    comment = comment.strip(" ")
    comment = re.sub(r'[?|$|.|!_:")(-+,]','',comment)
    return comment

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    # remove whitspace
    strOut = strOut.strip()
    # 
    strOut = re.sub('\s+',' ',strOut)
    return strOut


## Remote stop words, stemming, and tokenizing

In [5]:
df['tweet'] = df['tweet'].apply(casefolding)
df['tweet'] = df['tweet'].apply(text_preproc)

sw = stopwords.words('indonesian')

#tokenized
df['tokenized_tweet'] = df.apply(lambda row: nltk.word_tokenize(row['tweet']), axis=1)

# apply stopword removal
df['tokenized_tweet'] = df.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

In [6]:
# perform stemming on 'sw'
df['stemmed'] = df['tokenized_tweet'].apply(stemming)

df.head(100)

,id,tweet,tweet_date_time,tokenized_tweet,stemmed
0,1354579947801759748,jangan mancing di air keruh pigaihinasukujawa ...,2021-01-28 07:00:01,"{httpstcogooflieq, pigaihinasukujawa, air, ker...",httpstcogooflieq pigaihinasukujawa air keruh m...
1,1354609891395182592,kritik itu baik untuk pemerintah kritik itu me...,2021-01-28 08:59:00,"{kesukuan, orang, menghina, kebijakan, program...",suku orang hina bijak program perintah kritik ...
2,1354610143015673856,natalius pigai ngajak gelut cah pigaihinasukuj...,2021-01-28 09:00:00,"{pigai, ☝️☝️☝️☝️, httpstcowyawjzgcxt, gelut, n...",pigai httpstcowyawjzgcxt gelut ngajak pigaihi...
3,1354610143019888644,natalius pigai telah menghina suku jawa dan in...,2021-01-28 09:00:00,"{menghina, kebencian, pigai, ras, jawa, antars...",hina benci pigai ras jawa antarsuku pigaihinas...
4,1354610143032451072,apa yang kalian tangkap dari pernyataan pigai ...,2021-01-28 09:00:00,"{pigai, divideo, pernyataan, pigaihinasukujawa...",pigai video nyata pigaihinasukujawa httpstcobp...
5,1354610143040741381,anjimm dah pigai ini otak taro mana pak pigaih...,2021-01-28 09:00:00,"{httpstcoydcrwwvra, pigai, berasa, ya, taro, h...",httpstcoydcrwwvra pigai asa ya taro hebat otak...
6,1354610143040839680,maunya apa ya ni orang pigaihinasukujawa https...,2021-01-28 09:00:00,"{orang, ya, maunya, ni, httpstcokurnyqi, pigai...",orang ya mau ni httpstcokurnyqi pigaihinasukujawa
7,1354610143040860164,merendahkan suatu etnis tertentu itu dengan ra...,2021-01-28 09:00:00,"{menghina, orang, pigai, httpstcoupgsigykh, me...",hina orang pigai httpstcoupgsigykh aku rendah ...
8,1354610143074283520,semenjak keluar dr komnas ham kok bg pigai ini...,2021-01-28 09:00:00,"{pigai, komnas, ya, bg, httpstcowpxzhqdk, prov...",pigai komnas ya bg httpstcowpxzhqdk provokator...
9,1354610143107969025,waaah pigaihinasukujawa padahal orang jawa itu...,2021-01-28 09:00:00,"{orang, murah, penyabar, httpstconzbzyyqavw, j...",orang murah sabar httpstconzbzyyqavw jawa waaa...


## Learning dataset
X --> Variabel yang jadi input
Y --> Target 

In [7]:
# load data
df_train = pd.read_csv('./train-data/train-data-labeled.csv',sep="\t",)
df_train.head()

,sentimen,Tweet
0,2,lagu bosan apa yang aku save ni huhuhuhuhuhuhu...
1,2,kita lanjutkan saja diam ini hingga kau dan ak...
2,1,doa rezeki tak putus inna haa zaa larizquna ma...
3,1,makasih loh ntar kita bagi hasil aku 99 9 sisa...
4,2,aku tak faham betul jenis orang malaysia yang ...


In [8]:
# array of text, tweet content
text = df_train['Tweet'].tolist()
print(len(text))

# memisahkan X dan Y
# 1 >> positif
# 2 >> negatif
# 0 >> netral
y = df_train["sentimen"]
y = to_categorical(y)
print(y)

df_train['sentimen'].value_counts()

10806
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


0    5327
2    2887
1    2592
Name: sentimen, dtype: int64

In [9]:
token = Tokenizer()
token.fit_on_texts(text)

# token.index_word
vocab = len(token.index_word) + 1

#encoding texts
encode_text = token.texts_to_sequences(text)
# print(encode_text)

max_kata = 100
X = pad_sequences(encode_text, maxlen=max_kata, padding="post")
# X

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40, test_size=0.3, stratify=y)

# Convert to array of numpy
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

### Building models
Learn about CNN -> convolutional neural network
    filters
    overtippin
    
Build model, calculate epoch, and saves model...

In [11]:
# model definitions
vec_size = 300
model = Sequential()
model.add(Embedding(vocab, vec_size, input_length=max_kata))
model.add(Conv1D(64, 8, activation="relu"))
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          6813000   
_________________________________________________________________
conv1d (Conv1D)              (None, 93, 64)            153664    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 46, 64)            0         
_________________________________________________________________
dropout (Dropout)            (None, 46, 64)            0         
_________________________________________________________________
dense (Dense)                (None, 46, 32)            2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 46, 32)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 46, 16)            5

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=num_of_epoch, validation_data=(X_test, y_test))

Epoch 1/16
237/237 [==============================] - 28s 114ms/step - loss: 1.0547 - accuracy: 0.4794 - val_loss: 1.0521 - val_accuracy: 0.4907
Epoch 2/16
237/237 [==============================] - 30s 126ms/step - loss: 1.0039 - accuracy: 0.5264 - val_loss: 0.9897 - val_accuracy: 0.5494
Epoch 3/16
237/237 [==============================] - 29s 123ms/step - loss: 0.8238 - accuracy: 0.6507 - val_loss: 0.9306 - val_accuracy: 0.5783
Epoch 4/16
237/237 [==============================] - 29s 123ms/step - loss: 0.5502 - accuracy: 0.7969 - val_loss: 0.9365 - val_accuracy: 0.5848
Epoch 5/16
237/237 [==============================] - 33s 140ms/step - loss: 0.3527 - accuracy: 0.8761 - val_loss: 1.0521 - val_accuracy: 0.5697
Epoch 6/16
237/237 [==============================] - 27s 111ms/step - loss: 0.2332 - accuracy: 0.9264 - val_loss: 1.0730 - val_accuracy: 0.5895
Epoch 7/16
237/237 [==============================] - 27s 116ms/step - loss: 0.1835 - accuracy: 0.9470 - val_loss: 1.1409 - val_ac

In [13]:
model.save(model_data_filename)

In [14]:
# loading and testing models
new_model = tf.keras.models.load_model(model_data_filename)

# display summary
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          6813000   
_________________________________________________________________
conv1d (Conv1D)              (None, 93, 64)            153664    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 46, 64)            0         
_________________________________________________________________
dropout (Dropout)            (None, 46, 64)            0         
_________________________________________________________________
dense (Dense)                (None, 46, 32)            2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 46, 32)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 46, 16)            5

In [15]:
def get_encode(x):
    x = token.texts_to_sequences(x)
    x = pad_sequences(x, maxlen = max_kata, padding='post')
    return x

In [16]:
x = ['saya sulit sukar senang alay yes']
x = get_encode(x)

new_model.predict_classes(x)

/Users/muhammadhaviz/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([1])